In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


### Uploading train data

In [94]:
train_tw = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

### Looking at the data

In [36]:
train_tw.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [80]:
train_tw.keyword.value_counts().index.to_list()

['fatalities',
 'deluge',
 'armageddon',
 'sinking',
 'damage',
 'harm',
 'body%20bags',
 'outbreak',
 'evacuate',
 'fear',
 'collided',
 'siren',
 'twister',
 'windstorm',
 'sinkhole',
 'sunk',
 'hellfire',
 'weapon',
 'weapons',
 'famine',
 'explosion',
 'whirlwind',
 'earthquake',
 'derailment',
 'wreckage',
 'collision',
 'flames',
 'wrecked',
 'oil%20spill',
 'flooding',
 'deaths',
 'derailed',
 'thunder',
 'ambulance',
 'drowned',
 'explode',
 'blaze',
 'fatal',
 'typhoon',
 'fire',
 'upheaval',
 'hurricane',
 'injury',
 'bombed',
 'panic',
 'pandemonium',
 'debris',
 'blizzard',
 'destroy',
 'sandstorm',
 'mudslide',
 'ruin',
 'quarantined',
 'fatality',
 'bioterror',
 'hostages',
 'police',
 'emergency',
 'curfew',
 'crush',
 'wreck',
 'burning%20buildings',
 'wounded',
 'danger',
 'catastrophe',
 'dust%20storm',
 'engulfed',
 'evacuated',
 'evacuation',
 'cliff%20fall',
 'screaming',
 'floods',
 'attack',
 'devastation',
 'refugees',
 'massacre',
 'nuclear%20reactor',
 'displa

In [81]:
train_tw.location.value_counts().index.to_list()

['USA',
 'New York',
 'United States',
 'London',
 'Canada',
 'Nigeria',
 'UK',
 'Los Angeles, CA',
 'India',
 'Mumbai',
 'Washington, DC',
 'Kenya',
 'Worldwide',
 'Australia',
 'Chicago, IL',
 'California',
 'Everywhere',
 'New York, NY',
 'California, USA',
 'Florida',
 'San Francisco',
 'United Kingdom',
 'Los Angeles',
 'Indonesia',
 'Washington, D.C.',
 'Toronto',
 'NYC',
 'Ireland',
 'Seattle',
 'Earth',
 'San Francisco, CA',
 'Chicago',
 'Texas',
 'London, England',
 'Sacramento, CA',
 'New York City',
 'London, UK',
 'Atlanta, GA',
 'ss',
 'Dallas, TX',
 'San Diego, CA',
 'US',
 'Denver, Colorado',
 'Manchester',
 '304',
 'World',
 'Nashville, TN',
 'South Africa',
 'Scotland',
 'Houston, TX',
 'Denver, CO',
 'Tennessee',
 'Seattle, WA',
 'Sydney',
 'Austin, TX',
 'worldwide',
 'Memphis, TN',
 'Pennsylvania, USA',
 'Massachusetts',
 'Morioh, Japan',
 'Portland, OR',
 'Oklahoma City, OK',
 'Planet Earth',
 'Global',
 'California, United States',
 'Colorado',
 'Orlando, FL',
 'A

In [ ]:
train_tw.head(10)

In [24]:
def remove_substring(string, mark):
    while True:
        try:
            start = string.index(mark)
        except ValueError:
            break
        prefix = string[: start]
        string = string[start:]
        try:
            suffix = string[string.index(' ') + 1:]
        except ValueError: 
            suffix = ''
        string = prefix + suffix
    return string

### Here we will remove from the tweets the strings '@' and 'http' with the text following them

In [12]:
def string_process(string):
    marks = ['@', 'http']
    for element in marks:
        string = remove_substring(string, element)
    return string

### We can try to remove from the tweets only the character '@' without touching the text following it:

In [22]:
def remove_signs(string):
    prefixes = []
    while True:
        prefix, _, string = string.partition('@')
        prefixes.append(prefix)
        if string == '':
            break
    string = ''.join(prefixes)  # Here we can also try space ' ' instead of ''.
    string = remove_substring(string, 'http')
    return string

In [91]:
train_tw.text = train_tw.text.apply(remove_signs)

### We transform the type of all the cells in the columns 'keyword' and 'location' to str.

In [92]:
train_tw.keyword = train_tw['keyword'].astype('str')
train_tw.location = train_tw['location'].astype('str')

### In one case, we will try to remove from the train data observations without a specified keyword.

In [84]:
train_tw = train_tw[train_tw['keyword'] != 'nan']

### Let us remove all the strings which don't have the target value:

In [96]:
train_tw.dropna(subset='target')

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [98]:
train_tw

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
